In [2]:
pip install pandas numpy azure-storage-blob openpyxl


Note: you may need to restart the kernel to use updated packages.


In [4]:
from azure.storage.blob import BlobServiceClient
import pandas as pd
import io

# Connection string from Azure portal
conn_str = "DefaultEndpointsProtocol=https;AccountName=yashdatalake975;AccountKey=o83A5HSast67/VEqthSqB9ruitlvFCNjmlMPbH/o1hcxn0nwxjnWnaxTqFXJakDpqvC39Mjb18ls+ASt2Hspfg==;EndpointSuffix=core.windows.net"
container_name = "raw-data"
blob_name = "2019-Nov.csv"

# Connect to Blob
blob_service_client = BlobServiceClient.from_connection_string(conn_str)
blob_client = blob_service_client.get_container_client(container_name)

# Download blob
blob_data = blob_client.download_blob(blob_name)
data = pd.read_csv(io.BytesIO(blob_data.readall()))

print(data.head())


                event_time event_type  product_id   category_id  \
0  2019-11-01 00:00:00 UTC       view     1003461  2.053010e+18   
1  2019-11-01 00:00:00 UTC       view     5000088  2.053010e+18   
2  2019-11-01 00:00:01 UTC       view    17302664  2.053010e+18   
3  2019-11-01 00:00:01 UTC       view     3601530  2.053010e+18   
4  2019-11-01 00:00:01 UTC       view     1004775  2.053010e+18   

               category_code   brand   price    user_id  \
0     electronics.smartphone  xiaomi  489.07  520088904   
1  appliances.sewing_machine  janome  293.65  530496790   
2                        NaN   creed   28.31  561587266   
3  appliances.kitchen.washer      lg  712.87  518085591   
4     electronics.smartphone  xiaomi  183.27  558856683   

                           user_session  
0  4d3b30da-a5e4-49df-b1a8-ba5943f1dd33  
1  8e5f4f83-366c-4f70-860e-ca7417414283  
2  755422e7-9040-477b-9bd2-6a6e8fd97387  
3  3bfb58cd-7892-48cc-8020-2f17e6de6e7f  
4  313628f1-68b8-460d-84f6-cec7a

In [16]:
# Convert date to datetime format
data['event_time'] = pd.to_datetime(data['event_time'])

# Filter for only last year’s data
data_recent = data[data['event_time'].dt.year == 2019]

In [18]:
# Convert DataFrame to CSV string
csv_data = data_recent.to_csv(index=False)

# Upload back to blob
output_blob_client = blob_service_client.get_blob_client(container=container_name, blob="cleaned_data.csv")
output_blob_client.upload_blob(csv_data, overwrite=True)

{'etag': '"0x8DDD570C8092B54"',
 'last_modified': datetime.datetime(2025, 8, 7, 5, 10, 52, tzinfo=datetime.timezone.utc),
 'content_md5': None,
 'content_crc64': bytearray(b'\xa8\xe4YB\xc5\x06\xfbe'),
 'client_request_id': 'e3c1d111-734c-11f0-aca4-ea420ede37d1',
 'request_id': '19072bf6-501e-004a-6459-07e888000000',
 'version': '2025-07-05',
 'version_id': None,
 'date': datetime.datetime(2025, 8, 7, 5, 10, 52, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None}